# pattern for annex in table format

In [30]:
#table annex with numbers
import pandas
import os
import requests
import re
from bs4 import BeautifulSoup

url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32011R1002'
response = requests.get(url)
print(response.status_code)
data = response.text
data

200


'<?xml version="1.0" encoding="UTF-8"?>\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<!-- CONVEX # converter_version:9.15.0 # generated_on:20240305-0225 # ELI version:0.10 -->\r\n<head><meta name="format-detection" content="telephone=no"/><meta http-equiv=\'X-UA-Compatible\' content=\'IE=edge\'/><meta name=\'viewport\' content=\'width=device-width, initial-scale=1\'><meta name=\'WT.z_usr_lan\' content=\'EN\'>\n   <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n   <script type="text/javascript" src="/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_10289240325103055.js" data-dtconfig="app=47d4c64c3b67ec69|agentId=1e2d85f6ff8ee95d|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1720184838465|tp=500,50,0|agentUri=/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/eurlex-frontoffice/rb_39a3e95b-5423-482c-879

In [105]:
#get the document informatio

url= re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
celex= re.findall(r'uri=CELEX%([^"]+)', data)


In [111]:
# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=True)  # Extracting the identifying information
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)
    
    sanction_list.append({
        'URL': url,
        'CELEX':celex,
        'law_number' :title[0],
        'Publication Date': publication_date,
        'legal_base' : title[2],
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing
    })

# Display the results
for person in sanction_list:
    print(person)


{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011R1002'], 'CELEX': ['3A32011R1002'], 'law_number': '\n            COUNCIL IMPLEMENTING REGULATION (EU) No 1002/2011', 'Publication Date': ['12.10.2011'], 'legal_base': 'implementing Article 12(1) of Regulation (EU) No 359/2011 concerning restrictive measures directed against certain persons, entities and bodies in view of the situation in Iran', 'Name': 'ABBASZADEH-MESHKINI, Mahmoud', 'Identifying Information': '', 'Reason': 'Interior Ministry’s political director. As Head of the Article 10 Committee of the Law on Activities of Political Parties and Groups he is in charge of authorising demonstrations and other public events and registering political parties In 2010, he suspended the activities of two reformist political parties linked to Mousavi – the Islamic Iran Participation Front and the Islamic Revolution Mujahedeen Organization. From 2009 onwards he has consistently and continuously refused all non-go

In [14]:

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Extract additional document details using regex with checks
url_match = re.search(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"',data)
url = url_match.group(1) if url_match else None

publication_date_match = re.search(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
publication_date = publication_date_match.group(1) if publication_date_match else None

date_match = re.search(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>',data)
date = date_match.group(1) if date_match else None

title_match = re.search(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>',data)
title = title_match.group(1) if title_match else None

law_number_match = re.search(r'<p class="oj-hd-oj">C (\d+/\d+)</p>',data)
law_number = law_number_match.group(1) if law_number_match else None

case_number_match = re.search(r'Case\sT-\d+/\d+', data)
case_number = case_number_match.group(0) if case_number_match else None

legal_base_match = re.search(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
legal_base = legal_base_match.group(0) if legal_base_match else None

# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=True)  # Extracting the identifying information
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)

    sanction_list.append({
        'URL': url,
        'Publication Date': publication_date,
        'Date': date,
        'Title': title,
        'Law Number': law_number,
        'Case Number': case_number,
        'Legal Base': legal_base,
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing
    })

# Display the results
import json
print(json.dumps(sanction_list, indent=4))


[
    {
        "URL": "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011R1002",
        "Publication Date": "12.10.2011",
        "Date": null,
        "Title": "ANNEX",
        "Law Number": null,
        "Case Number": null,
        "Legal Base": null,
        "Name": "ABBASZADEH-MESHKINI, Mahmoud",
        "Identifying Information": "",
        "Reason": "Interior Ministry\u2019s political director. As Head of the Article 10 Committee of the Law on Activities of Political Parties and Groups he is in charge of authorising demonstrations and other public events and registering political parties In 2010, he suspended the activities of two reformist political parties linked to Mousavi \u2013 the Islamic Iran Participation Front and the Islamic Revolution Mujahedeen Organization. From 2009 onwards he has consistently and continuously refused all non-governmental gatherings, therefore denying a constitutional right to protest and leading to many arrests of peaceful d